In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

datasets = {}

In [3]:
# Parse data from Shiller (1870 - now)
# http://www.econ.yale.edu/~shiller/data.htm
data = pd.read_csv("data/SP500-Shiller.csv", delimiter='\t')
data['Stock yield'] = data['P'].pct_change().fillna(0.0)
data['Div yield'] = (data['D'] / data['P']).fillna(0.0) / 12 # Shiller uses div yield of trailing 12 months
data['Inflation'] = data['CPI'].pct_change().fillna(0.0)
data['Yield'] =  data['Stock yield'] + data['Div yield']
datasets['Shiller'] = data[['Date', 'Div yield', 'Stock yield', 'Yield', 'Inflation']]

print("Data size for Shiller's S&P 500 data is %d" % len(data.index))
datasets['Shiller'].tail(15)

Data size for Shiller's S&P 500 data is 1773


,Date,Div yield,Stock yield,Yield,Inflation
1758,2017.07,0.001614,0.008262,0.009876,-0.000690
1759,2017.08,0.001623,0.000864,0.002487,0.002994
1760,2017.09,0.001610,0.014909,0.016519,0.005295
1761,2017.10,0.001578,0.025738,0.027316,-0.000632
1762,2017.11,0.001564,0.014318,0.015882,0.000024
1763,2017.12,0.001530,0.027271,0.028801,-0.000588
1764,2018.01,0.001472,0.047089,0.048561,0.005448
1765,2018.02,0.001529,-0.030339,-0.028810,0.004535
1766,2018.03,0.001542,-0.000883,0.000658,0.002261
1767,2018.04,0.001581,-0.018181,-0.016601,0.003975


In [4]:
dataset = datasets['Shiller']

def simulate_etf_buy(p, capital_start, etf_price_start, contribution, months, etf_ter=0.0025, etf_dividend_tax_leakage=0.1, dividend_cost_fn=None):
    sample = dataset.iloc[p:p+months]
    
    etf_price = etf_price_start
    etfs = (capital_start - capital_start % etf_price) / etf_price
    cash = capital_start % etf_price
    
    dividends = 0
    for i in sample.index:
        # Step 1: We add the monthly contribution to our cash pool.
        cash += contribution
        
        # Step 2: ETF increases in price...
        etf_gain = etf_price * sample.at[i, 'Stock yield']
        # ... and costs are subtracted.
        etf_price = (etf_price + etf_gain) * (1 - etf_ter / 12)
        
        # Step 3: Accumulate dividends...
        dividends += etf_price * sample.at[i, 'Div yield']
        if i % 3 == 0:
            # ... and pay them out every quarter...
            gross_dividend = etfs * dividends
            # ... subtracting unrecoverable dividend taxes (tax leakage)...
            net_dividend = gross_dividend * (1 - etf_dividend_tax_leakage)
            if dividend_cost_fn:
                # ... and dividend processing fees if applicable.
                net_dividend = net_dividend - dividend_cost_fn(net_dividend)
            cash += net_dividend
            dividends = 0
        
        # Step 4: Buy ETFs from the available cash.
        etfs = etfs + (cash - cash % etf_price) / etf_price
        cash = cash % etf_price
    
    return cash + etfs * etf_price

def degiro_custody_dividend_cost(net_dividend):
    return min(net_dividend * 0.1, 1 + net_dividend * 0.03)

def simulate_etf_buys(initial_capital, initial_etf_price, monthly_contribution, duration_in_years=30, etf_ter=0.0025, etf_dividend_leakage=0.1, dividend_cost_fn=None):
    months = duration_in_years * 12
    index = np.arange(0, len(dataset) - months)
    results = pd.Series((simulate_etf_buy(p, initial_capital, initial_etf_price, monthly_contribution, months, etf_ter, etf_dividend_leakage, dividend_cost_fn) for p in index), index=index, name='end_value')
    return results

In [5]:
# Simulation for VWRL with startprice of 74.55 (August 2018) and monthly contribution of 100 euros for 50 years.
vwrl_start_price = 74.55
start_cash = 0
monthly_contribution = 100
num_years = 50

%timeit -n1 -r1 res = simulate_etf_buys(start_cash, vwrl_start_price, monthly_contribution, num_years, dividend_cost_fn=degiro_custody_dividend_cost)

7.55 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
